# Imports

In [3]:
#------For upcoming development--------

#!pip install --upgrade agno
#!pip install pytesseract
#!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib



#------For examples----------

#!pip install duckduckgo-search
#!pip install yfinance
#!pip install duckduckgo_search

In [4]:
#Imports for actual project

import os
import re
import gc
import sys
import time
import json
import shutil
import base64
import signal
import pickle
import inspect
import asyncio
import logging
import argparse
import datetime
import tempfile
import pytesseract


from agno.agent import Agent
from agno.team.team import Team
from agno.models.nebius import Nebius

from openai import OpenAI
from bs4 import BeautifulSoup
from email.mime.text import MIMEText
from jinja2 import Environment, BaseLoader

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

In [5]:
#For examples, not the actual project
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.yfinance import YFinanceTools
from textwrap import dedent

# Initialize API keys & Model Categories (Classifiers vs. Readers vs. Reasoners)

Nebius has good security credentials, but there is an option to run models on a larger local computer (would likely only be feasible to run under 100b-param models, and even then that's veryintensive), Ollama could work for this in shell 

[Note: I just found out there is Agno documentation for Ollama, so I wouldn't need a shell workaround].

In [16]:
#Nebius server stuff, let's me run models that are way bigger than I could reasonably run on my computer.
nebius_api_key = "XXXXXXXXXXXXXXXXXX"
classifier_llm = "Qwen/Qwen2.5-32B-Instruct" #This model has been tested to be best performing in this context.
reader_llm = "Qwen/Qwen2.5-32B-Instruct" #This model HAS NOT been tested to be best performing in this context.
reasoner_llm = "Qwen/Qwen2.5-72B-Instruct" #This model HAS NOT been tested to be best performing in this context.

# Single Agent Example

In [7]:
email_processor = Agent(
    model = Nebius(api_key = nebius_api_key, id = classifier_llm),
    instructions="You are Edgar Allen Poe, write what is requested of you.",
    show_tool_calls=True,
    markdown=True,
    debug_mode=True,
)

# Print the response in the terminal
answer = email_processor.print_response("write a two sentence horror story")
print(str(answer))

DEBUG ****** Agent ID: 075026d7-0d37-4d6e-92a3-0b506a26dd1e ******

DEBUG ***** Session ID: 60b0a13f-7741-4299-8864-421c74bae4b3 *****

DEBUG ** Agent Run Start: d97cb721-4c3a-4cb8-b542-f7e2293f875e ***

DEBUG ------------------ Nebius Response Start -------------------

DEBUG ------------- Model: Qwen/Qwen2.5-32B-Instruct -------------

...

DEBUG ************************  METRICS  *************************

DEBUG ------------------- Nebius Response End --------------------

DEBUG Added RunResponse to Memory

DEBUG *** Agent Run End: d97cb721-4c3a-4cb8-b542-f7e2293f875e ****

None


# Team Example

In [22]:
web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    model=Nebius(api_key=nebius_api_key, id=classifier_llm),
    tools=[DuckDuckGoTools()],
    instructions=dedent("""\
        You are an experienced web researcher and news analyst! 🔍
        Follow these steps when searching for information:
        1. Start with the most recent and relevant sources
        2. Cross-reference information from multiple sources
        3. Prioritize reputable news outlets and official sources
        4. Always cite your sources with links
        5. Focus on market-moving news and significant developments
        Your style guide:
        - Present information in a clear, journalistic style
        - Use bullet points for key takeaways
        - Include relevant quotes when available
        - Specify the date and time for each piece of news
        - Highlight market sentiment and industry trends
        - End with a brief analysis of the overall narrative
        - Pay special attention to regulatory news, earnings reports, and strategic announcements\
    """),
    show_tool_calls=True,
    markdown=True,
    add_name_to_instructions=True,
)

finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model=Nebius(api_key=nebius_api_key, id=classifier_llm),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions=dedent("""\
        You are a skilled financial analyst with expertise in market data! 📊
        Follow these steps when analyzing financial data:
        1. Start with the latest stock price, trading volume, and daily range
        2. Present detailed analyst recommendations and consensus target prices
        3. Include key metrics: P/E ratio, market cap, 52-week range
        4. Analyze trading patterns and volume trends
        5. Compare performance against relevant sector indices
        Your style guide:
        - Use tables for structured data presentation
        - Include clear headers for each data section
        - Add brief explanations for technical terms
        - Stay professional
        - Use bullet points for quick insights
        - Compare current values with historical averages
        - End with a data-driven financial outlook\
    """),
    show_tool_calls=True,
    markdown=True,
    add_name_to_instructions=True,  # Added this based on the example
)

agent_team = Team(
    name="Investment Report Team",
    mode="collaborate", #Gives semant
    members=[web_agent, finance_agent],
    model=Nebius(api_key=nebius_api_key, id=reasoner_llm),
    instructions=dedent("""\
        You are the lead editor of a prestigious financial news desk! 📰
        Your role:
        1. Coordinate between the web researcher and financial analyst
        2. Combine their findings into a compelling narrative
        3. Ensure all information is properly sourced and verified
        4. Present a balanced view of both news and data
        5. Highlight key risks and opportunities
        Your style guide:
        - Start with an attention-grabbing headline
        - Begin with a powerful executive summary
        - Present financial data first, followed by news context
        - Use clear section breaks between different types of information
        - Include relevant charts or tables when available
        - Add 'Market Sentiment' section with current mood
        - Include a 'Key Takeaways' section at the end
        - End with 'Risk Factors' when appropriate
        - Sign off with 'Market Watch Team' and the current date\
    """),
    success_criteria="The team has compiled a comprehensive investment report on the requested stock ticker.",  
    enable_agentic_context=True, 
    add_datetime_to_instructions=True,
    show_tool_calls=True,
    markdown=True,
    show_members_responses=True,  
)

# Just call print_response directly - no asyncio needed
agent_team.print_response(
    "Use the team to write a report on McDonalds Corp (MCD)",
    stream=True,
    stream_intermediate_steps=True
)


Output()

WARNING  Could not run function set_shared_context(state=MCD, report_date=2025-07-18)

ERROR    1 validation error for Team.get_set_shared_context_function.<locals>.set_shared_context                   
         report_date                                                                                               
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value='2025-07-18', input_type=str]
             For further information visit https://errors.pydantic.dev/2.10/v/unexpected_keyword_argument          
         Traceback (most recent call last):                                                                        
           File "/opt/anaconda3/lib/python3.13/site-packages/agno/tools/function.py", line 635, in execute         
             result = self.function.entrypoint(**arguments)                                                        
           File "/opt/anaconda3/lib/python3.13/site-packages/pydantic/_internal/_validate_call.py", line 38, in    
         wrapper_function                                                                                          
             return wrapper(*args, **kwargs)                                                                       
           File "/opt/anaconda3/lib/python3.13/site-packages/pydantic/_internal/_validate_call.py", line 111, in   
         __call__                                                                                                  
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
         pydantic_core._pydantic_core.ValidationError: 1 validation error for                                      
         Team.get_set_shared_context_function.<locals>.set_shared_context                                          
         report_date                                                                                               
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value='2025-07-18', input_type=str]
             For further information visit https://errors.pydantic.dev/2.10/v/unexpected_keyword_argument

/opt/anaconda3/lib/python3.13/site-packages/agno/tools/duckduckgo.py:90: RuntimeWarning: This package 
(`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs = DDGS(

/opt/anaconda3/lib/python3.13/site-packages/agno/tools/duckduckgo.py:71: RuntimeWarning: This package 
(`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs = DDGS(

/opt/anaconda3/lib/python3.13/site-packages/agno/tools/duckduckgo.py:71: RuntimeWarning: This package 
(`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs = DDGS(